# 1. 학습 테스트 데이터 로딩

## 1.1 남자

In [ ]:
import pandas as pd
df_m = pd.read_csv('man_cluster.txt')

In [ ]:
df_m.info

In [ ]:
df_m

In [ ]:
# class_mode = categorical을 적용하기위한 작업
df_m['class'] = df_m['class'].apply(lambda x :[x])

In [ ]:
df_m.reset_index(drop=True,inplace=True)

In [ ]:
df_m

## 1.2 여자

In [ ]:
df_w = pd.read_csv('woman_cluster.txt')

In [ ]:
# class_mode = categorical을 적용하기위한 작업
df_w['class'] = df_w['class'].apply(lambda x :[x])

In [ ]:
df_w.reset_index(drop=True,inplace=True)

In [ ]:
pd.set_option('max_rows', 50)

In [ ]:
df_w

# 2. 모델 생성

In [ ]:
import matplotlib.pyplot as plt
from keras import models
from keras import layers

model = models.Sequential()

# hidden layer에 합성곱과 맥스 풀링 설정
model.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))


# 1차원으로 변환
model.add(layers.Flatten())

# 입력 데이터에 50%의 노드를 무작위로 사용하지 않게 막는다.
model.add(layers.Dropout(0.5))

# 결과얻기
# class를 6개로 정하였으므로 softmax 설정
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(6, activation='softmax'))

In [ ]:
# from keras.models import Sequential
# from keras_preprocessing.image import ImageDataGenerator
# from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
# from keras.layers import Conv2D, MaxPooling2D
# from keras import regularizers, optimizers
# import pandas as pd
# import numpy as np

In [ ]:
# model = Sequential()
# model.add(Conv2D(256, (3, 3), padding='same',
#                  input_shape=(256,256,3)))
# model.add(Activation('relu'))
# model.add(Conv2D(128, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Conv2D(64, (3, 3), padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(512))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(6, activation='softmax'))

# 3. 모델 컴파일

In [ ]:
import tensorflow as tf
from keras import optimizers
from keras import losses
from keras import metrics

In [ ]:
# #출력이 13개이므로 categorical_crossentropy 설정
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6),
#               loss='categorical_crossentropy',
#               metrics=['acc'])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

# 4. 이미지 증식

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# 4.1.1 train,validation, test datagen 생성(남자)
>  8:1:1의 비율

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    shear_range = 0.2,
    horizontal_flip=True,
    fill_mode='constant'
)

# 검증 데이터는 증식을 사용하면 안됨
# 모델의 과적합을 막기 위한 기술이므로 검증에서는 당연히 배제되어야 함.
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    #이미지 경로
    df_m[:6000],
    x_col='imagepath',
    y_col='class',
    # 사용할 이미지 크기
    target_size=(128,128),
    # 배치 크기
    batch_size = 2,
    # 손실 데이터 연관
    class_mode = 'categorical',
#     classes = ['0','1','2','3','4','5','6','7','8','9','10','11','12']
)

In [ ]:
validation_generator = test_datagen.flow_from_dataframe(
    #이미지 경로
    df_m[6000:6795],
    x_col='imagepath',
    y_col= 'class',
    # 사용할 이미지 크기
    target_size=(128,128),
    # 배치 크기
    batch_size = 2,
    # 손실 데이터 연관
    class_mode = 'categorical',
#     classes = ['0','1','2','3','4','5','6','7','8','9','10','11','12']
)

In [ ]:
test_generator = test_datagen.flow_from_dataframe(
    #이미지 경로
    df_m[6795:],
    x_col='imagepath',
    y_col='class',
    # 사용할 이미지 크기
    target_size=(128,128),
    # 배치 크기
    batch_size = 2,
    # 손실 데이터 연관
    class_mode = None,
)

# 4.1.2 train,validation, test datagen 생성(여자)
>  8:1:1의 비율

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    shear_range = 0.2,
    horizontal_flip=True,
    fill_mode='constant'
)

# 검증 데이터는 증식을 사용하면 안됨
# 모델의 과적합을 막기 위한 기술이므로 검증에서는 당연히 배제되어야 함.
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    #이미지 경로
    df_w[:6000],
    x_col='imagepath',
    y_col='class',
    # 사용할 이미지 크기
    target_size=(128,128),
    # 배치 크기
    batch_size = 1,
    # 손실 데이터 연관
    class_mode = 'categorical',
#     classes = ['0','1','2','3','4','5','6','7','8','9','10','11','12']
)

In [ ]:
validation_generator = test_datagen.flow_from_dataframe(
    #이미지 경로
    df_w[6000:6850],
    x_col='imagepath',
    y_col= 'class',
    # 사용할 이미지 크기
    target_size=(128,128),
    # 배치 크기
    batch_size = 1,
    # 손실 데이터 연관
    class_mode = 'categorical',
#     classes = ['0','1','2','3','4','5','6','7','8','9','10','11','12']
)

In [ ]:
test_generator = test_datagen.flow_from_dataframe(
    #이미지 경로
    df_w[6850:],
    x_col='imagepath',
    y_col='class',
    # 사용할 이미지 크기
    target_size=(128,128),
    # 배치 크기
    batch_size = 1,
    # 손실 데이터 연관
    class_mode = None,
)

# 5. 모델 학습

In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')

print('Found GPU at : ', device_name)

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

In [ ]:
import tensorboard
from datetime import datetime

log_dir="logs\\fit\\" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
with tf.device('/device:GPU:0'):
    history = model.fit_generator(generator=train_generator,
                        steps_per_epoch=STEP_SIZE_TRAIN,
                        validation_data=validation_generator,
                        validation_steps=STEP_SIZE_VALID,
                        epochs=10
    )

# 6. 모델 시각화

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

epochs = range(len(acc))

plt.plot(epochs, acc , 'b', label = 'Training acc',color='orange')
plt.plot(epochs, val_acc, 'b',label='Validation acc',color='blue')
plt.legend()

plt.show()

In [ ]:
import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))

plt.plot(epochs, loss , 'b', label = 'Training loss',color='orange')
plt.plot(epochs, val_loss, 'b',label='Validation loss',color='blue')
plt.legend()

plt.show()

In [ ]:
model.

In [ ]:
from tensorflow.keras.models import load_model
model.save("model_class6_man.h5")

In [ ]:
from tensorflow.keras.models import load_model
model.save("model_class6_woman.h5")

In [ ]:
from tensorflow.keras.models import load_model
model=load_model("model_center_class9.h5")

In [ ]:
model.save_weights('checkpoints')

In [ ]:
from tensorflow.keras.models import load_model
model.save("model_center_class9.h5")

In [ ]:
from tensorflow.keras.models import load_model
model=load_model("model_class6_man.h5")

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model

history_path = './model/center_class6_extra_history.npy'
np.save(history_path, history.history)

In [ ]:
test_df = pd.DataFrame(columns=['imagepath'])
test_df.loc[0,'imagepath'] = 'C:\\Users\\성수현\\Desktop\\590.jpg'
test_df

In [ ]:
test_generator = test_datagen.flow_from_dataframe(
    #이미지 경로
    test_df,
    x_col='imagepath',
    # 사용할 이미지 크기
    target_size=(128,128),
    # 배치 크기
    batch_size = 1,
    # 손실 데이터 연관
    class_mode = None,
)

In [ ]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=1,verbose=1)

In [ ]:
def hangulFilePathImageRead(filePath): 
    stream = open(filePath , "rb") 
    bytes = bytearray(stream.read()) 
    numpyArray = np.asarray(bytes, dtype=np.uint8) 
    return cv2.imdecode(numpyArray , cv2.IMREAD_COLOR) 

In [ ]:
import cv2

In [ ]:
img=hangulFilePathImageRead('C:\\Users\\성수현\\Desktop\\590.jpg')

print(pred[0].argmax())
plt.imshow(img)